In [ ]:
import scipy as sp
import numpy as np
import porespy as ps
import matplotlib.pyplot as plt
import scipy.io as sio
import seaborn as sns
import skimage as ski

imageSize = 250

primaryImage = np.load('subBeadPackPy250_justSpheres.npy')
secondaryImage = np.load('finalSimFile3D250.npy')
primaryImage[primaryImage == 255] = 1

velSecondaryMat = sio.loadmat('velNormSecondary.mat')
velDataNormSecondary = velSecondaryMat['velNorm']

velPrimaryMat = sio.loadmat('velNormPrimary.mat')
velDataNormPrimary = velPrimaryMat['velNorm']

resolution = 16.81E-6 # adding resolution in meters

# Plot pore space and velocity

Start with sample with secondary porosity

In [ ]:

slice = 49

secondaryImage = np.transpose(secondaryImage)
primaryImage = np.transpose(primaryImage)

fig, (p1, p2) = plt.subplots(1, 2)

fig.suptitle('Pore space and velocity map')
p1.imshow(velDataNormSecondary[:,:,slice])
p2.imshow(secondaryImage[:,:,slice])

Now plot the sample with just primary porosity

In [ ]:
fig, (p1, p2) = plt.subplots(1, 2)

fig.suptitle('Pore space and velocity map')
p1.imshow(velDataNormPrimary[:,:,slice])
p2.imshow(primaryImage[:,:,slice])



Now compare the pore space of both samples

In [ ]:
slice = 49

fig, (p1, p2) = plt.subplots(1, 2)

fig.suptitle('Pore space comparision')
p1.imshow(primaryImage[:,:,slice])
p2.imshow(secondaryImage[:,:,slice])

In [ ]:
# testIM = primaryImage.astype(bool)
# testIM = testIM[:,:,slice]
# skelIM = ski.morphology.skeletonize(testIM)
#
# plt.imshow(skelIM)

# Extract pore network information

In [ ]:
snowFiltSecondary = ps.filters.snow_partitioning(im=secondaryImage,r_max=4,sigma=0.4, return_all=True)
poreInfoSecondary = ps.networks.regions_to_network(snowFiltSecondary.regions, dt=snowFiltSecondary.dt)

nRegions = np.unique(snowFiltSecondary.regions).size
secondaryPoreDiamVecSecondary = np.zeros(nRegions,)

for a in range(0, poreInfoSecondary['pore.diameter'].size):
    secondaryPoreDiamVecSecondary[a] = poreInfoSecondary['pore.diameter'][a]

In [ ]:
snowFiltPrimary = ps.filters.snow_partitioning(im=primaryImage,r_max=4,sigma=0.4, return_all=True)
poreInfoPrimary = ps.networks.regions_to_network(snowFiltPrimary.regions, dt=snowFiltPrimary.dt)

nRegions = np.unique(snowFiltPrimary.regions).size
primaryPoreDiamVec = np.zeros(nRegions,)

for a in range(0, poreInfoPrimary['pore.diameter'].size):
    primaryPoreDiamVec[a] = poreInfoPrimary['pore.diameter'][a]

In [ ]:
slice = 19

imSub = primaryImage[:,:,slice]
snowFiltSub = ps.filters.snow_partitioning(im=imSub,r_max=5,sigma=0.4, return_all=True)
poreInfoSub = ps.networks.regions_to_network(snowFiltSub.regions, dt=snowFiltSub.dt)
test = ps.metrics.regionprops_3D(snowFiltSub.regions)
regionTest = snowFiltSub.regions
subsubTest = regionTest[regionTest == 8]

plt.imshow(regionTest)

# Plot velocity histogram for simulation

In [ ]:
allPrimaryVelocities = np.ndarray.flatten(velDataNormPrimary)

filtIndex = allPrimaryVelocities > 0
trueZeroIndex = allPrimaryVelocities == 0

filtPrimaryVelocities = allPrimaryVelocities[filtIndex]
filtZeroPrimaryVelocities = allPrimaryVelocities[trueZeroIndex]

fig, axes = plt.subplots(1, 1, figsize=(18, 10))
sns.distplot(filtPrimaryVelocities, ax=axes, hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
axes.set_title('Primary porosity ')
axes.set_xlabel('All pore velocities',fontsize='x-large')

In [ ]:
allSecondaryVelocities = np.ndarray.flatten(velDataNormSecondary)

filtIndex = allSecondaryVelocities > 0
trueZeroIndex = allSecondaryVelocities == 0

filtSecondaryVelocities = allSecondaryVelocities[filtIndex]
filtZeroSecondaryVelocities = allSecondaryVelocities[trueZeroIndex]

fig, axes = plt.subplots(1, 1, figsize=(18, 10))
sns.distplot(filtSecondaryVelocities, ax=axes, hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
axes.set_title('Secondary porosity ')
axes.set_xlabel('All pore velocities',fontsize='x-large')




In [ ]:
allPrimaryVelocities = np.ndarray.flatten(velDataNormPrimary)
allSecondaryVelocities = np.ndarray.flatten(velDataNormPrimary)

fig, axes = plt.subplots(1, 2, figsize=(18, 10))
sns.distplot(allPrimaryVelocities, ax=axes[0], hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
axes[0].set_title('Primary porosity ')
axes[0].set_xlabel('All pore velocities',fontsize='x-large')

sns.distplot(allSecondaryVelocities, ax=axes[1], hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
axes[1].set_title('Secondary porosity')
axes[1].set_xlabel('All pore velocities',fontsize='x-large')

# Skeleton implementation


In [ ]:
primaryRegionData = ps.metrics.regionprops_3D(snowFiltPrimary.regions) #Gives properties for each region
primaryRegionData.append([])

primaryRegions = snowFiltPrimary.regions

In [ ]:
secondaryRegionData = ps.metrics.regionprops_3D(snowFiltSecondary.regions) #Gives properties for each region
secondaryRegionData.append([])

secondaryRegions =  snowFiltSecondary.regions

On primary image

In [ ]:
cubeSize = len(primaryImage)
primarySkelImage = np.zeros(primaryImage.shape)
edgeImage = np.zeros(primaryImage.shape)
visit = np.zeros(len(primaryRegionData))


for a in range(0,cubeSize):
    for b in range(0,cubeSize):
        for c in range(0, cubeSize):
            regionLabel = snowFiltPrimary.regions[a,b,c]
            if regionLabel != 0: # Don't want grains to be counted
                regionLabel = regionLabel -1 # Adjusting index to work with region props command
                if visit[regionLabel] == 0:
                    visit[regionLabel] = 1
                    #index = snowFiltPrimary.regions[a,b,c]
                    regionInd = snowFiltPrimary.regions == regionLabel
                    regionBorder = primaryRegionData[regionLabel].slice
                    primarySkelImage[regionBorder[0],regionBorder[1],regionBorder[2]] = primaryRegionData[regionLabel].skeleton
                    #edgeImage[snowFiltPrimary.regions[a,b,c] == regionLabel] = regionData[regionLabel].border

In [ ]:
plt.imshow(primarySkelImage[:,:,40])
ps.io.to_vtk(primarySkelImage,'PS_skeleton')

#primaryImage[primaryImage == 1] = 255 # Make red/blue distinction
#ps.io.to_vtk(primaryImage,'primaryImage')

On secondary image

In [ ]:
cubeSize = len(secondaryImage)
secondarySkelImage = np.zeros(secondaryImage.shape)
edgeImage = np.zeros(secondaryImage.shape)
visit = np.zeros(len(secondaryRegionData))

for a in range(0,cubeSize):
    for b in range(0,cubeSize):
        for c in range(0, cubeSize):
            regionLabel = snowFiltSecondary.regions[a,b,c]
            if regionLabel != 0: # Don't want grains to be counted
                regionLabel = regionLabel -1 # Adjusting index to work with region props command
                if visit[regionLabel] == 0:
                    visit[regionLabel] = 1
                    #index = snowFiltPrimary.regions[a,b,c]
                    regionInd = snowFiltPrimary.regions == regionLabel
                    regionBorder = secondaryRegionData[regionLabel].slice
                    secondarySkelImage[regionBorder[0],regionBorder[1],regionBorder[2]] = secondaryRegionData[regionLabel].skeleton


# Now plot velocitites on skeleton

In [ ]:
primaryVelocitiesSkeleton = []

for a in range(0,cubeSize):
    for b in range(0,cubeSize):
        for c in range(0, cubeSize):
            if primarySkelImage[a,b,c] != 0:
                primaryVelocitiesSkeleton = np.append(primaryVelocitiesSkeleton,velDataNormPrimary[a,b,c])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 10))
sns.distplot(primaryVelocitiesSkeleton, ax=axes[0], hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

sns.distplot(primaryVelocitiesSkeleton[primaryVelocitiesSkeleton != 0], ax=axes[1], hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})


In [ ]:
secondaryVelocitiesSkeleton = []

for a in range(0,cubeSize):
    for b in range(0,cubeSize):
        for c in range(0, cubeSize):
            if secondarySkelImage[a,b,c] != 0:
                secondaryVelocitiesSkeleton = np.append(secondaryVelocitiesSkeleton,velDataNormSecondary[a,b,c])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 10))
sns.distplot(secondaryVelocitiesSkeleton, ax=axes[0], hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

sns.distplot(secondaryVelocitiesSkeleton[secondaryVelocitiesSkeleton != 0], ax=axes[1], hist=True, kde=True,
             bins=int(80), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})




Look at number of zeros in each image

In [ ]:
primaryVelZeros = primaryVelocitiesSkeleton[primaryVelocitiesSkeleton == 0]
secondaryVelZeros = secondaryVelocitiesSkeleton[secondaryVelocitiesSkeleton == 0]

primaryVelFilt = primaryVelocitiesSkeleton[primaryVelocitiesSkeleton != 0]
secondaryVelFilt = secondaryVelocitiesSkeleton[secondaryVelocitiesSkeleton != 0]

In [ ]:
z